In [1]:
import pandas as pd
from collections import Counter
import statistics

In [2]:
df = pd.read_csv('https://firebasestorage.googleapis.com/v0/b/civic-abode-211408.appspot.com/o/hotels.csv?alt=media&token=aa9c8eac-6a57-4caa-bfa4-e5a3b198d2fd')

In [3]:
hotels = df.copy()
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [72]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386803 non-null  object        
 1   additional_number_of_scoring                386803 non-null  int64         
 2   review_date                                 386803 non-null  datetime64[ns]
 3   average_score                               386803 non-null  float64       
 4   hotel_name                                  386803 non-null  object        
 5   reviewer_nationality                        386803 non-null  object        
 6   negative_review                             386803 non-null  object        
 7   review_total_negative_word_counts           386803 non-null  int64         
 8   total_number_of_reviews                     386803 non-null  int64        

#### Filling the empty values

In [5]:
hotels['lat'] = hotels['lat'].fillna(statistics.median(hotels['lat']))
hotels['lng'] = hotels['lat'].fillna(statistics.median(hotels['lat']))


In [6]:

hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: x.split(' ')[0])
hotels['days_since_review'] = hotels['days_since_review'].astype(int)

In [7]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['review_date'].max()

Timestamp('2017-08-03 00:00:00')

In [16]:
hotels['hotel_address'][11].split(' ')[-2:]
hotels['hotel_country'] = hotels['hotel_address'].apply(lambda x: ' '.join(x.split(' ')[-2:]) if 'United Kingdom' in x else x.split(' ')[-1])
hotels['hotel_city'] = hotels['hotel_address'].apply(lambda x: ''.join(x.split(' ')[-5]) if 'United Kingdom' in x else x.split(' ')[-2])


In [35]:
hotels['tags'][386799]

"[' Leisure trip ', ' Couple ', ' Standard Double Room ', ' Stayed 4 nights ', ' Submitted from a mobile device ']"

In [71]:
hotels['stayed_nights'] = hotels['tags'].str.extract(r'(\d+(?= night))')
hotels['stayed_nights'] = hotels['stayed_nights'].fillna(hotels['stayed_nights'].median()).astype(int)

In [81]:
hotels['tags']

0         [' Leisure trip ', ' Couple ', ' Studio Suite ...
1         [' Business trip ', ' Couple ', ' Standard Dou...
2         [' Leisure trip ', ' Solo traveler ', ' Modern...
3         [' Leisure trip ', ' Solo traveler ', ' Standa...
4         [' Business trip ', ' Couple ', ' Standard Dou...
                                ...                        
386798    [' Leisure trip ', ' Group ', ' Club Double or...
386799    [' Leisure trip ', ' Couple ', ' Standard Doub...
386800    [' Business trip ', ' Solo traveler ', ' Singl...
386801    [' Leisure trip ', ' Solo traveler ', ' Deluxe...
386802    [' Leisure trip ', ' Couple ', ' Bunk Bed Room...
Name: tags, Length: 386803, dtype: object

#### Сколько уникальных тегов представлено в наборе данных?

In [121]:
tags_list = (hotels['tags'].apply(lambda x: x.replace("' ", '').replace(" '", '')[1:-1].split(', '))).tolist()
all_tags = []
[all_tags.extend(x) for x in tags_list]
len(set(all_tags))

2368

#### Какой тег представлен в наибольшем числе отзывов?

In [122]:
all_tags
counts = Counter(all_tags)
counts.most_common()[0]

('Leisure trip', 313593)

#### Из тегов выясните, на сколько ночей чаще всего останавливаются путешественники в отелях.

In [124]:
counts.most_common()[3]

('Stayed 1 night', 145373)

In [112]:
# hotels = hotels.drop(['hotel_address', 'review_date', 'hotel_name', 'reviewer_nationality', 'negative_review', 'positive_review',
#                       'tags', 'days_since_review'], axis=1)
# hotels['lat'] = hotels['lat'].fillna(hotels['lat'].median())
# hotels['lng'] = hotels['lng'].fillna(hotels['lng'].median())

In [132]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [135]:
# непрерывные признаки
num_cols = ['total_number_of_reviews', 'review_total_negative_word_counts', 'review_total_positive_word_counts']

# категориальные признаки
cat_cols = ['average_score', 'lat', 'lng', 'days_since_review', 'tags', 'reviewer_nationality', 'additional_number_of_scoring']

In [136]:
y=y.astype('int')


from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

ValueError: could not convert string to float: '531 day'

In [114]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [21]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [23]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14135472138000876


Небольшой бонус:


In [ ]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)